In [3]:
import numpy as np

N = 16
S = 12 # first test simple case when N == S
# S = 10 # number of slices / CT-images / scans

r = np.random.random((S, N, N))
# r = np.load('r.npy')
ro  = 1e-5 + 1.24e-3*r - 2.83e-7*r*r + 2.79e-11*r*r*r
c = (ro + 0.112) * 1.38e-6

# тут не факт что все 1
l = 1 # dt
h = 1 # dx = dy = dz

K = l / h * c
K2 = K**2
K2_3_ro = K2 / 3 / ro


# initial conditions
P_pp = np.random.random((S, N, N)) # previous previous t - 2
P_p  = np.random.random((S, N, N)) # previous          t - 1

In [4]:
def old_slow1(P_pp, P_p):
    '''slow explicit n^3, c-style implementation v0.2'''
    
    S = P_p.shape[0]
    N = P_p.shape[1]

    # P[2:-2, 2:-2, 2:-2] = (2 - 7.5 * K2[2:-2, 2:-2, 2:-2]) * P_p[2:-2, 2:-2, 2:-2] - P_pp[2:-2, 2:-2, 2:-2]
    # P = np.zeros_like(P_p)

    P = (2 - 7.5 * K2) * P_p - P_pp


    for i in range(2, S - 2):
        for j in range(2, N - 2):
            for k in range(2, N - 2):
                s1 = (P_p[i + 1, j    , k    ] + \
                      P_p[i - 1, j    , k    ] + \
                      P_p[i    , j + 1, k    ] + \
                      P_p[i    , j - 1, k    ] + \
                      P_p[i    , j    , k + 1] + \
                      P_p[i    , j    , k - 1]
                )
                P[i, j, k] += 4/3 * K2[i, j, k] * s1

                s2 = (P_p[i + 2, j    , k    ] + \
                      P_p[i - 2, j    , k    ] + \
                      P_p[i    , j + 2, k    ] + \
                      P_p[i    , j - 2, k    ] + \
                      P_p[i    , j    , k + 2] + \
                      P_p[i    , j    , k - 2]
                )
                P[i, j, k] -= K2[i, j, k] / 12 * s2

                s3 = ((P_p[i + 1, j    , k    ] - P_p[i-1 , j  , k  ]) - (P_p[i + 2, j, k] - P_p[i - 2, j, k]) / 8) * (ro[i + 1, j, k] - ro[i - 1, j, k])
                s4 = ((P_p[i    , j + 1, k    ] - P_p[i   , j-1, k  ]) - (P_p[i, j + 2, k] - P_p[i, j - 2, k]) / 8) * (ro[i, j + 1, k] - ro[i, j - 1, k])
                s5 = ((P_p[i    , j    , k + 1] - P_p[i-1 , j  , k-1]) - (P_p[i, j, k + 2] - P_p[i, j, k - 2]) / 8) * (ro[i, j, k + 1] - ro[i, j, k - 1])
                P -= K2_3_ro[i, j, k] * s3
                P -= K2_3_ro[i, j, k] * s4
                P -= K2_3_ro[i, j, k] * s5
    return P[2:-2, 2:-2, 2:-2]

In [5]:
%timeit old_slow1(P_pp, P_p)

35.3 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
